## 1. langchain's experimental implementation

references
- [docs](https://api.python.langchain.com/en/latest/experimental_api_reference.html#module-langchain_experimental.plan_and_execute)
- [github](https://github.com/langchain-ai/langchain/tree/master/libs/experimental/langchain_experimental/plan_and_execute)

### default prompts

#### config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
from langchain_community.chat_models import BedrockChat

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    # model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "max_tokens": 4096, 
        "temperature": 0.5,
    },
    streaming=False,
    region_name="us-west-2",
)

#### define planner

In [3]:
from langchain_experimental.plan_and_execute.planners.chat_planner import load_chat_planner

planner = load_chat_planner(
    llm=llm,
    # system_prompt=planner_system_prompt
)

In [4]:
question = "Who is the current president of the United States? What is their current age raised to the second power?"
planner_results = planner.plan({
    "input": question
})

In [5]:
print(planner_results)

steps=[Step(value='Identify the current president of the United States'), Step(value='Find the current age of the president'), Step(value="Raise the president's age to the second power (square it)"), Step(value="Given the above steps taken, please respond to the user's original question\n\n")]


In [6]:
print(planner_results.steps[0])

value='Identify the current president of the United States'


#### define executor

In [7]:
from langchain_experimental.plan_and_execute.executors.agent_executor import load_agent_executor
from utils.tools import tools
# make sure to only select below tools and comment rest all
# - search-wikipedia, search-arxiv, wolfram-aplha. 

executor = load_agent_executor(
    llm=llm,
    tools=tools,
    verbose=True,
    include_task_in_prompt=True,
)

In [8]:
from langchain_experimental.plan_and_execute.schema import (
    BaseStepContainer,
    ListStepContainer,
)
from langchain_experimental.pydantic_v1 import Field

executor_results = executor.step({
    "previous_steps": Field(default_factory=ListStepContainer),
    "current_step": planner_results.steps[0],
    "objective": question
})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search-wikipedia",
  "action_input": "Current President of the United States"
}
```


Observation: Page: President of the United States
Summary: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is one of the world's most powerful political figures and the leade

In [9]:
print(executor_results)

response='The current president of the United States is Joe Biden. His current age (81) raised to the second power is 6,561.'


#### plan-and-execute: combine above

In [10]:
from langchain_experimental.plan_and_execute.agent_executor import PlanAndExecute

plan_and_execute = PlanAndExecute(
    planner=planner,
    executor=executor
)

In [11]:
question = "Who is the current president of the United States? What is their current age raised to the second power?"
plan_and_execute_results = plan_and_execute.invoke({
    "input": question
})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search-wikipedia",
  "action_input": "President of the United States"
}
```


Observation: Page: President of the United States
Summary: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is one of the world's most powerful political figures and the leader of the

In [12]:
print(plan_and_execute_results)

{'input': 'Who is the current president of the United States? What is their current age raised to the second power?', 'output': 'The current president of the United States is Joe Biden. His current age of 81 years old raised to the second power is 6,561.'}


### with custom prompts

In [ ]:
# from utils.react_prompt import *
# from utils.tools import tools

# # make sure to only select below tools and comment rest all
# # - search-wikipedia, search-arxiv, wolfram-aplha. 
# prompt = create_agent_prompt(tools)
# # prompt.pretty_print()

# # TODO: doubt if I need chat_history
# executor_chain = (
#     {
#         "question": lambda x: x["question"],
#         "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
#         "chat_history": lambda x: x["chat_history"]
#     }
#     | prompt
#     | llm.bind(stop=["\nObservation"])
#     | ReActSingleInputOutputParser()
#     # | CustomOutputParser()
# )

In [18]:
from langchain.agents.structured_chat.prompt import FORMAT_INSTRUCTIONS, PREFIX, SUFFIX

# print(PREFIX)
# print(FORMAT_INSTRUCTIONS)
# print(SUFFIX)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:


In [19]:
from langchain import hub

prompt = hub.pull("hwchase17/structured-chat-agent")

prompt.pretty_print()

================================ System Message ================================

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

==============

## 2. custom